# [COMPAS] Baseline -- Disparate Mistreatment

In [1]:
import os,sys

sys.path.append('../../../fair-classification_python3/fair_classification') # the code for fair classification is in this directory
sys.path.append("../")
cwd = '../../../core'
sys.path.append(cwd)


import numpy as np
import pandas as pd
import utils as ut
import funcs_disp_mist as fdm

from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve
from sklearn.impute import KNNImputer, SimpleImputer

from IPython.display import Markdown, display
import matplotlib.pyplot as plt


from load_compas import * 
from missing_module import * 


In [3]:
X, y, x_control = load_compas_data()

df = pd.DataFrame(X, columns= ['age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 
                               'priors_count', 'c_charge_degree'])

idxx = df[df['race']==0].index
print(idxx[:10])

y = pd.Series(y, name="two_year_recid")
### IMPORTANT: y must be +1 or -1 for DCCP to work ###
# y[y==-1] = 0

df = pd.concat([df, y], axis=1)
df_bal = balance_data(df, 'race', 0)


s = 777
df_ms = generate_missing(df_bal, c_label='race', ms_label='sex', p_ms0=0.4, p_ms1=0.1, seed=s)
df_ms = generate_missing(df_ms, c_label='race', ms_label='priors_count', p_ms0=0.6, p_ms1=0.2, seed=s)


privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

df_ms.groupby(df_ms['race']).mean()


Number of people recidivating within two years
-1    2795
 1    2483
dtype: int64


('Features we will be using for classification are:', ['age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree'], '\n')
Int64Index([0, 1, 2, 8, 10, 13, 14, 15, 21, 22], dtype='int64')


,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,sex,priors_count,c_charge_degree,two_year_recid
race,,,,,,,
0.0,0.592011,0.147408,0.260580,0.820127,0.134560,0.308131,0.067998
1.0,0.536377,0.298621,0.165002,0.771791,-0.245055,0.408464,-0.218260


In [4]:

fair = 'fnr'
tau_list = [0.5, 1, 2, 5, 10, 20]

fr_mean, acc_mean, fr_std, acc_std = [], [], [], [] 


for tau in tau_list: 
    fr_list = []
    acc_list = [] 
    
    for seed in range (1, 11): 
        ################## Train-Test Split ################### 
        dataset_orig_train, dataset_orig_test = train_test_split(df_ms, test_size=0.3, random_state=seed)

        ########################################################

        ##################### Imputation ###################### 
        
        ## Change the following two lines to get mean or k-nn results ##
#         imputer = SimpleImputer()
        imputer = KNNImputer()
        imputer.fit(dataset_orig_train)
        df_train = pd.DataFrame(imputer.transform(dataset_orig_train), columns=dataset_orig_train.columns, 
                                                  index=dataset_orig_train.index)

        df_test = pd.DataFrame(imputer.transform(dataset_orig_test), columns=dataset_orig_test.columns, 
                                                  index=dataset_orig_test.index)

        ########################################################
        sensitive_attrs = ['race']

        X_train = df_train.iloc[:,:-1]
        X_train['intercept'] = np.ones(len(X_train))
        x_control_train = dict({'race': np.array([int(x) for x in X_train['race']])})
        X_train = np.array(X_train.drop(columns=['race']))
        y_train = np.array(df_train.iloc[:,-1])

        print(X_train.shape)

        X_test = df_test.iloc[:,:-1]
        X_test['intercept'] = np.ones(len(X_test))
        x_control_test = dict({'race': np.array([int(x) for x in X_test['race']])})
        X_test = np.array(X_test.drop(columns=['race']))
        y_test = np.array(df_test.iloc[:,-1])


        ##########################################################

        ################ Disparate Mistreatment ################## 
        if fair == 'fpr':
            cons_type = 1 
        elif fair == 'fnr': 
            cons_type = 2
        elif fair == 'acc':
            cons_type = 0 
        elif fair == 'eqodds':
            cons_type = 4 

        mu = 1.2
        loss_function = "logreg" # perform the experiments with logistic regression
        EPS = 1e-6

        sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}} } # zero covariance threshold, means try to get the fairest solution
        cons_params = {"cons_type": cons_type, 
                       "tau": tau, 
                       "mu": mu, 
                       "sensitive_attrs_to_cov_thresh": sensitive_attrs_to_cov_thresh}

        w = fdm.train_model_disp_mist(X_train, y_train, x_control_train, loss_function, EPS, cons_params)
        train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w, X_train, y_train, x_control_train, X_test, y_test, x_control_test, sensitive_attrs)

        fr = np.abs(s_attr_to_fp_fn_test['race'][0][fair] - s_attr_to_fp_fn_test['race'][1][fair])
        fr_list.append(fr)
        acc_list.append(test_score)
        
    fr_mean.append(np.mean(fr_list))
    fr_std.append(np.std(fr_list))
    acc_mean.append(np.mean(acc_list))
    acc_std.append(np.std(acc_list))
    

(2944, 7)


Accuracy: 0.537
||  s  || FPR. || FNR. ||
||  0  || 0.92 || 0.03 ||
||  1  || 0.74 || 0.08 ||


(2944, 7)


Accuracy: 0.574
||  s  || FPR. || FNR. ||
||  0  || 0.90 || 0.04 ||
||  1  || 0.65 || 0.11 ||


(2944, 7)


Accuracy: 0.541
||  s  || FPR. || FNR. ||
||  0  || 0.90 || 0.04 ||
||  1  || 0.69 || 0.13 ||


(2944, 7)


Accuracy: 0.517
||  s  || FPR. || FNR. ||
||  0  || 0.93 || 0.02 ||
||  1  || 0.74 || 0.10 ||


(2944, 7)


Accuracy: 0.526
||  s  || FPR. || FNR. ||
||  0  || 0.94 || 0.02 ||
||  1  || 0.74 || 0.10 ||


(2944, 7)


Accuracy: 0.519
||  s  || FPR. || FNR. ||
||  0  || 0.79 || 0.10 ||
||  1  || 0.75 || 0.21 ||


(2944, 7)


Accuracy: 0.544
||  s  || FPR. || FNR. ||
||  0  || 0.90 || 0.03 ||
||  1  || 0.69 || 0.11 ||


(2944, 7)


Accuracy: 0.529
||  s  || FPR. || FNR. ||
||  0  || 0.92 || 0.03 ||
||  1  || 0.70 || 0.16 ||


(2944, 7)


Accuracy: 0.550
||  s  || FPR. || FNR. ||
||  0  || 0.87 || 0.02 ||
||  1  || 0.72 || 0.12 ||


(2944, 7)


Accuracy: 0.517


In [5]:
fr_mean

[0.09205802604766725,
 0.09628800964087007,
 0.09380110561444607,
 0.09921148958668606,
 0.11379087648002646,
 0.21995954023921688]

In [6]:
acc_mean

[0.535419968304279,
 0.5389064976228208,
 0.5378763866877971,
 0.5328843106180667,
 0.5431061806656101,
 0.6141838351822504]

In [7]:
with open('knn_disp_mstrtment_result.pkl', 'wb+') as f: 
    pickle.dump({'fr_mean': fr_mean, 'fr_std': fr_std, 'acc_mean': acc_mean, 'acc_std': acc_std}, f)